In [ ]:
import pandas as pd

test_file = "common-sense/test_data.csv"
df_test = pd.read_csv(test_file)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

PATH_TO_DIR = "models/bert"
CHECKPOINT = "checkpoint-1000"
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMultipleChoice.from_pretrained(f"{PATH_TO_DIR}/{MODEL_NAME}/{CHECKPOINT}/")

In [ ]:
options = ["OptionA", "OptionB", "OptionC"]
answers = ['A', 'B', 'C']

def predict(example):
    prompts = []
    for opt in options:
        prompts.append([example['FalseSent'], example[opt]])

    inputs = tokenizer(prompts, return_tensors="pt", padding=True)
    labels = torch.tensor(0).unsqueeze(0)
    
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
    logits = outputs.logits
    predicted_class = logits.argmax().item()

    return answers[predicted_class]

df_test['label'] = df_test.apply(lambda row: predict(row), axis=1)

In [ ]:
submission_df = df_test[["id", "label"]].copy()
submission_df = submission_df.rename(columns={"label": "answer"})
submission_df.set_index("id", inplace=True)
submission_df.to_csv(f"submissions/Submission_10-12-2022_1.csv")